# 초기세팅

## 모듈 임포트

In [19]:
from typing import List, Dict, Optional, TypedDict, Any
from fastapi import FastAPI , Body ,HTTPException, APIRouter, Query 
import torch
import ast
import json
import uuid
import numpy as np
import psycopg2
from pgvector.psycopg2 import register_vector
from sentence_transformers import SentenceTransformer
from google.genai import types
from fastapi.middleware.cors import CORSMiddleware
import os
from langgraph.graph import StateGraph, END

## 모델 로딩

### 재미나이 로딩

In [20]:
# API 키 로딩
keys = []
def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                keys.append(value)

# API 키 로드 및 환경변수 설정
load_api_keys('api_key.txt')

In [21]:
def keyChange():
    global client
    from google import genai
    tt = keys.pop(0)
    keys.append(tt)
    client = genai.Client(api_key=tt)

keyChange() #무료 키 다썻으면 다른 키로 로딩하기

### 백터DB & 임베딩 모델 로딩

In [22]:
model_vector = SentenceTransformer('jhgan/ko-sroberta-multitask')
# 2. 원격 pgvector 연결 설정
def get_db_connection():
    db_info= dict()
    with open("db_info.txt","r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=",1)
                db_info[key] = value

    conn = psycopg2.connect(
        host=db_info['host'],
        database=db_info['database'],
        user=db_info['user'],
        password=db_info['password'],
        port=db_info['port']
    )
    register_vector(conn)
    return conn


## State

In [24]:
class State(TypedDict, total = False):
    txt: str  #front
    ep_num : int  #front 
    subtitle : str #front
    check : dict
    work_id : int    #front
    user_id : str    #front
    인물 : dict
    세계 : dict
    장소 : dict
    사건 : dict
    물건 : dict
    집단 : dict
    


# 기능

## 백터 DB

### 임베딩 백터 : 택스트 데이터를 임베딩 벡터로 바꿔서 반환(768차원)

In [25]:
def get_vector(text):
    return model_vector.encode([text])[0]

### 백터 DB 검색(유사도)  
(카테고리 or *(전부), 유저 질문, user_id, work_id, 유사도 몇까지 출력할지, 몇개 출력할지)

In [26]:

def userQuestion(category,user_query,user_id,work_id,sim,limit):  # 태그(*,인물,장소,사건,세계,등)
    conn = get_db_connection()
    cur = conn.cursor()
    query_vector = get_vector(user_query)
    search_sql = """
    SELECT * FROM (
        SELECT 
            id,
            setting, 
            ep_num,
            1 - (embedding <=> %s::vector) AS similarity 
        FROM lorebooks 
        WHERE embedding IS NOT NULL 
            AND %s = ANY(user_id)
            AND work_id = %s
            AND (%s = '*' OR category = %s)
    ) sub
    WHERE similarity >= %s  -- 여기서 원하는 유사도 문턱값을 설정 (예: 0.3 이상)
    ORDER BY similarity DESC  
    LIMIT %s;
    """
    cur.execute(search_sql, (query_vector,user_id,work_id,category,category,sim,limit))
    result = cur.fetchall()
    cur.close()
    conn.close()
    return result

In [27]:
# userQuestion("인물","럭산나라는 이름을 가진 인물","eZHXUSS8",5,0,10)

### DB 검색(일반)  
(카테고리, 검색할 대상, user_id, work_id, 유사도 몇까지 출력할지, 몇개 출력할지)

In [28]:
def select(category,keyword,user_id,work_id): #태그,키워드,타이틀,작가 4개 다있어야함
    conn = get_db_connection()
    cur = conn.cursor()
    search_sql= """
    SELECT 
        id,
        setting,
        ep_num
    FROM lorebooks 
    WHERE %s = ANY(user_id)
        AND work_id = %s
        AND category = %s
        AND keyword = %s
    """
    cur.execute(search_sql, (user_id,work_id,category,keyword))
    result = cur.fetchall()
    # print(result)
    cur.close()
    conn.close()
    return result

### DB 업로드(설정집 신규)

In [29]:
def lorebooks_insert(universe_id,work_id,user_id,setting):
    conn = get_db_connection()
    cur = conn.cursor()
    for tag, values in setting.items():
        for val in values:
            if not val:
                continue
            ep_num = val.pop('ep_num',[])
            keyword,data = val.popitem()
            embd = get_vector(f'{keyword} : {data}')
            try:
                insert_sql = """
                    INSERT INTO lorebooks ( universe_id, work_id, user_id, keyword, category, ep_num, setting, embedding)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                    """
                cur.execute(insert_sql, (
                        universe_id, 
                        work_id,
                        [user_id],
                        keyword,
                        tag,
                        ep_num,
                        json.dumps({keyword:data}), 
                        embd # psycopg2가 리스트 형태를 자동으로 vector 타입으로 변환합니다.
                    ))
                conn.commit()
                print("성공적으로 저장되었습니다!")
            except Exception as e:
                print(e)
                return "500 Internal Server Error"  # DB연결실패
    cur.close()
    conn.close()

# 수동 신규 설정집 업로드
def db_insert(universe_id, work_id, user_id, keyword, category, ep_num, setting):
    embd = get_vector(f'{setting}')
    conn = get_db_connection()
    cur = conn.cursor()
    insert_sql = """
    INSERT INTO lorebooks ( universe_id, work_id, user_id, keyword, category, ep_num, setting, embedding)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    cur.execute(insert_sql, (
        universe_id,
        work_id,
        user_id,
        keyword,
        category,
        ep_num,
        json.dumps(setting),
        embd # psycopg2가 리스트 형태를 자동으로 vector 타입으로 변환합니다.
    ))
    conn.commit()
    cur.close()
    conn.close()
    print("성공적으로 저장되었습니다!")

### DB 수정(설정집 결합)

In [30]:
def lorebooks_update(setting):
    for lore_id,result,ep_num in setting:
        conn = get_db_connection()
        cur = conn.cursor()
        new_vector = f"{result}"
        embedding_vector = get_vector(new_vector)
        update_sql = """
        UPDATE lorebooks 
        SET 
            ep_num = %s,
            setting = %s,
            embedding = %s
        WHERE id = %s;
        """
        try:
            cur.execute(update_sql, (ep_num,json.dumps(result), embedding_vector, lore_id))
            conn.commit()
            print(f"데이터가 성공적으로 수정되었습니다.")
        except Exception as e:
            conn.rollback()
            print(e)
            return (f"수정 실패: {e}")
        finally:
            cur.close()
            conn.close()


# DB 수동 수정
def db_update(lore_id,universe_id, work_id, user_id, keyword, category, ep_num, setting):
    embd = get_vector(f'{setting}')
    conn = get_db_connection()
    cur = conn.cursor()
    embedding_vector = get_vector(new_vector)
    update_sql = """
    UPDATE lorebooks 
    SET 
        universe_id = %s,
        work_id = %s,
        user_id = %s,
        keyword = %s,
        category = %s,
        ep_num = %s,
        setting = %s,
        embedding = %s
    WHERE id = %s;
    """
    try:
        cur.execute(update_sql, (universe_id, work_id, user_id, keyword, category, ep_num, json.dumps(setting), embd, lore_id))
        conn.commit()
        print(f"데이터가 성공적으로 수정되었습니다.")
    except Exception as e:
        conn.rollback()
        return (f"수정 실패: {e}")
    finally:
        cur.close()

## LLM 

### 메시지 입력

In [70]:
count_change= 0
models = ["gemini-2.5-flash-lite", 'gemini-3-flash',"gemini-2.5-flash","gemma-3-27b-it"]
def modelchange():
    global mod
    mod = models.pop(0)
    models.append(mod)
modelchange()    
def genai(system_prompt,msg):
    global count_change
    try:
        if mod == "gemma-3-27b-it":
            response = client.models.generate_content(
                model=mod,
                contents=system_prompt+msg,
            )
            return response.text
        else:
            response = client.models.generate_content(
                model= mod,
                config=types.GenerateContentConfig(
                    system_instruction=system_prompt, # 여기에 시스템 프롬프트를 넣습니다.
                    temperature=0.1, # 데이터 추출이므로 낮게 설정
                ),
                contents=msg
            )
            return response.text
    except Exception as e:
        count_change +=1
        if count_change >=30:
            modelchange()   
        keyChange()
        return genai(system_prompt,msg)


### 카테고리 추출

In [86]:
def Categories(state: State) -> State:
    system_prompt="""### Role
너는 소설 원문에서 설정 데이터를 누락 없이 추출하여 구조화하는 전문 데이터 분석가입니다.

### Task
당신은 '본문'의 글에서 설정집을 생성하기 쉽도록 키워드를 추출합니다. 
추출할 정보는 '인물, 세계, 장소, 사건, 물건, 집단'에 관한 내용이며, 설정집에 필요한 필수적인 정보들만 추출할 것입니다.
이를 딕셔너리{} 형태로 구조화하세요.

### Extraction Guidelines (카테고리별 정의)
- **인물** 
이름이 언급되거나 고유한 역할이 있는 등장인물(단순 엑스트라 제외)
- **세계**
마법 체계, 정치 시스템, 신화적 배경, 물리적 법칙 등 세계관을 이해하기에 필수적인 설정과 개념
- **장소**
사건이 벌어지는 지리적 배경, 건물 이름, 특정 지역
- **사건**
스토리에 영향을 끼칠 수 있는 구체적인 변화, 전투, 조약, 과거의 사건 등
- **물건**
고유 명칭이 있는 도구, 무기, 유물, 중요한 상징적 아이템
- **집단**
가문, 국가, 단체 등 2인 이상의 고유명칭을 가진 결사체

### Constraints
- 특정 주인공 중심의 요약을 하지 마세요. 등장하는 키워드를 분류하세요.
- 결과물에는 불릿 포인트, 볼드체, 추가 설명을 절대 포함하지 말고 순수 JSON/Dict 형식만 출력하세요.
- **본문에 명확히 나오지 않은 정보는 빈 배열 `[]`로 처리하세요.
- **수식어 제거**: 대상을 특정하기 위한 수식어(예: '데마시아의 정치체계','총의 악마','OO왕국 기사단')를 제외한 수식어는(예: '비시르 함장' X, '비시르' O) 모두 제거하고 핵심 키워드만 추출하세요.
- **개체명 표준화: 동일 대상을 지칭하는 이명(예: '몰락한 왕'과 '비에고','비에고 국왕'과 '비에고' > 두 경우 다 '비에고'로 출력)은 가장 공식적인 명칭 하나로 통합하세요.

### Output Format 
{
'인물':['인물1','인물2',...],
'세계':['규칙1','규칙2',...],
'장소':['장소1','장소2'],
'사건':['사건1','사건2'],
'물건':['물건1',....],
'집단':['집단1','집단2'],
}

"""
    msg = f"본문: {state.get('txt','')} \n 위 본문을 분석하여 '인물, 세계, 장소, 사건, 물건, 집단' 정보를 추출해줘."
    
    result = genai(system_prompt,msg)
    print("목록 추출 완료")
    print("=="*8)
    print(result)
    print("=="*8)
    try:
        st = result.index('{')
        ed = result.index('}')
        result = ast.literal_eval(result[st:ed+1])
    except:
        print("딕셔너리 변환 에러")
        # state = Categories(state)
    return {**state,
            'check' : result
           }

### 1.설정 추출 - 인물

In [144]:
def charter(state: State) -> State:
    lis = state.get('check')
    lis = lis['인물']
    txt = state.get('txt')
    chart = state.get('인물',{})
    for i in range(0, len(lis), 2):
        chunk = lis[i:i+2]  # 예: ['인물1', '인물2']
        names_str = ", ".join(chunk)
        system_prompt = f"""# Role
귀하는 소설의 원문을 분석하여 인물 설정집(Character Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, 미리 추출한 등장인물명 [{names_str}] 각각의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
새로운 정보가 발견될 때마다 기존 설정과 대조할 수 있도록 매우 구체적이고 객관적인 수치와 묘사 위주로 추출해야 합니다.
각 인물별로 독립된 키를 가진 딕셔너리 형태로 출력해야 합니다
본문에서 해당 정보가 있다면 "Unknown"을 삭제하고 정보를 str형식으로 적으세요
# 본문
{txt}

# extraction_rules
1. **Fact-Based**: 추측하지 말고 본문에 명시된 텍스트 그대로를 기반으로 추출하세요.
"""+"""   
# JSON_Output_Format
{ "인물명1" : 
    { "별명": "본문에 명시된 별명 (없으면 Unknown)",
      "배경": "간략한 배경 스토리(없으면 Unknown)",
      "종족": "종족 (없으면 Unknown)",
      "연령": "나이 또는 연령대 (없으면 Unknown)",
      "직업/신분": "신분 (없으면 Unknown)",
      "소속집단/가문": "소속 (없으면 Unknown)",
      "외형": "외모 묘사 (없으면 Unknown)",
      "성격": ["성격 키워드1 (없으면 Unknown)"],
      "기술/능력": [
        {"기술명": "능력 이름 (없으면 Unknown)", "숙련도/강도": "수준 (없으면 Unknown)", "상세효과": "효과설명 (없으면 Unknown)"}
      ],
      "인물관계": [
        {"대상이름": "상대방 이름(없으면 Unknown)", "관계": "관계 정의( 없으면 Unknown"), "상세내용": "관계 설명(없으면 Unknown)"}
      ]
    },
    "인물명2" : 
    { "별명": "본문에 명시된 별명 (없으면 Unknown)",
      "배경": "간략한 배경 스토리(없으면 Unknown)",
      "종족": "종족 (없으면 Unknown)",
      "연령": "나이 또는 연령대 (없으면 Unknown)",
      "직업/신분": "신분 (없으면 Unknown)",
      "소속집단/가문": "소속 (없으면 Unknown)",
      "외형": "외모 묘사 (없으면 Unknown)",
      "성격": ["성격 키워드1 (없으면 Unknown)"],
      "기술/능력": [
        {"기술명": "능력 이름 (없으면 Unknown)", "숙련도/강도": "수준 (없으면 Unknown)", "상세효과": "효과설명 (없으면 Unknown)"}
      ],
      "인물관계": [
        {"대상이름": "상대방 이름(없으면 Unknown)", "관계": "관계 정의( 없으면 Unknown"), "상세내용": "관계 설명(없으면 Unknown)"}
      ]
    }
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 만약 등장인물이 하나만 있을 경우에는 한 개체만 출력하세요.
- 제공한 등장인물 2명을 제외한 나머지는 추가적으로 출력하지 마세요."""
        
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식을 마크다운 없이 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)

            result = json.loads(result)
            print("변환 성공!")
            for k, v in result.items():
                chart[k] = v
        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
        
    return {**state,
           '인물' :chart
           }

### 2.설정추출 - 세계관

In [145]:
def rule(state: State) -> State:
    lis = state.get('check')
    lis = lis['세계']
    txt = state.get('txt')
    rule = state.get('세계',{})
    for i in range(0, len(lis), 4):
        chunk = lis[i:i+4]  # 예: ['인물1', '인물2']
        names_str = ", ".join(chunk)
        system_prompt = f"""# Role
귀하는 소설의 원문을 분석하여 세계관 설정집을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, 미리 추출한 세계관 키워드 [{names_str}] 각각의 정보을/를 본문 없이 작품 속 세계의 규칙을 이해할 수 있도록 300자 이내로 설명하는 문장을 만들어 JSON 포맷으로 출력하세요.

# 본문
{txt}

# extraction_rules
1. **Fact-Based**: 요약은 가능하지만 본문의 내용을 바탕으로 추측 없이 문장을 생성하세요.
2. 연관된 이야기 없이 대상에 대한 내용만으로 짧게 요약하여 문장을 생성하세요.

"""+"""   
# JSON_Output_Format
{
"세계관 키워드1" : "세계관 1 설명",
"세계관 키워드2" : "세계관 2 설명",....
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 본문에 명확히 나오지 않은 정보는 "Unknown" 또는 빈 배열 `[]`로 처리하세요.
- 만약 세계관 키워드가 하나만 있을 경우에는 한 세계관만 출력하세요.
- 제공한 키워드를 제외한 나머지는 추가적으로 출력하지 마세요.
"""
        
        msg= f"규칙을 바탕으로 순수한 Json 텍스트 형식을 마크다운 없이 출력하세요."           
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)

            result = json.loads(result)
            print("변환 성공!")
            for k, v in result.items():
                rule[k] = v
        except:
            print("딕셔너리 변환 에러")
            print("result")
            #state = charter(state)
    
    return {**state,
           '세계' :rule
           }

### 3.설정추출 - 장소

In [124]:
def place(state: State) -> State:
    lis = state.get('check')
    lis = lis['장소']
    txt = state.get('txt')
    place = state.get('장소',{})
    for i in range(0, len(lis), 3):
        chunk = lis[i:i+3]  
        names_str = ", ".join(chunk)
        system_prompt=  f"""# Role
귀하는 소설의 원문을 분석하여 장소 설정집(Place Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, 미리 추출한 장소 키워드 [{names_str}] 각각의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
새로운 정보가 발견될 때마다 기존 설정과 대조할 수 있도록 매우 구체적이고 객관적인 수치와 묘사 위주로 추출해야 합니다.
본문에서 해당 정보가 있다면 "Unknown"을 삭제하고 정보를 str형식으로 적으세요

# 본문
{txt}

# extraction_rules
1. **Fact-Based**: 추측하지 말고 본문에 명시된 텍스트 그대로를 기반으로 추출하세요.
"""+"""   
# JSON_Output_Format
{"장소 키워드1": 
    { "별칭": "지역별칭 (없으면 Unknown)"
      "분위기": "지역의 분위기(없으면 Unknown)",
      "역사": "지역의 역사 또는 생성된 시기 (없으면 Unknown)",
      "위치": "지역의 위치 (없으면 Unknown)",
      "집단": ["해당 공간에 속한 소속 집단/가문(없으면 Unknown)"],
      "하부지역": ["이 장소에 속한 다른 지역 (없으면 Unknown)"],
      "작중묘사" :["해당 장소에 대한 묘사 (없으면 Unknown)"]
    },
 "장소 키워드2": 
    { "별칭": "지역별칭 (없으면 Unknown)"
      "분위기": "지역의 분위기(없으면 Unknown)",
      "역사": "지역의 역사 또는 생성된 시기 (없으면 Unknown)",
      "위치": "지역의 위치 (없으면 Unknown)",
      "집단": ["해당 공간에 속한 소속 집단/가문(없으면 Unknown)"],
      "하부지역": ["이 장소에 속한 다른 지역 (없으면 Unknown)"],
      "작중묘사" :["해당 장소에 대한 묘사 (없으면 Unknown)"]
    }
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 본문에 정보가 전혀 없는 항목만 "Unknown"으로 남겨두세요.
- 만약 장소 키워드가 하나만 있을 경우에는 한 장소만 출력하세요.
- 제공한 키워드를 제외한 나머지는 임의로 추가하여 출력하지 마세요."""
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
                    

        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            for k, v in result.items():
                place[k] = v
        except:
            print("딕셔너리 변환 에러")
            print("result")
            #state = charter(state)
    
    return {**state,
           '장소' :place
           }

### 4.설정추출 - 사건

In [133]:
def event(state: State) -> State:
    lis = state.get('check')
    lis = lis['사건']
    txt = state.get('txt')
    event = state.get('사건',{})
    for i in range(0, len(lis), 5):
        chunk = lis[i:i+5]  
        names_str = ", ".join(chunk)
        system_prompt= f"""# Role
귀하는 소설의 원문을 분석하여 사건 설정집(Place Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, 미리 추출한 사건 키워드 [{names_str}] 각각의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
설명에는 해당 사건에 관해 간략한 원인과 결과를 한줄요약하여 써야한다

# 본문
{txt}
"""+"""   
# JSON_Output_Format
{
"사건 키워드1": "해당 사건에 관한 짧은 설명",
"사건 키워드2": "해당 사건에 관한 짧은 설명"
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 만약 키워드가 하나만 있을 경우에는 한 사건만 출력하세요.
- 제공한 키워드를 제외한 나머지는 임의로 추가하여 출력하지 마세요.
"""
        msg=f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            for k, v in result.items():
                event[k] = v
        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
    return {**state,
           '사건' :event
           }

### 5.설정추출 - 아이템

In [134]:
def item(state: State) -> State:
    lis = state.get('check')
    lis = lis['물건']
    txt = state.get('txt')
    item = state.get('물건',{})
    
    for i in range(0, len(lis), 4):
        chunk = lis[i:i+4]  
        names_str = ", ".join(chunk)
        system_prompt = f"""# Role
귀하는 소설의 원문을 분석하여 아이템 설정(item Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.
본문에서 해당 정보가 있다면 "Unknown"을 삭제하고 정보를 str형식으로 적으세요

# Task
제공된 소설 본문을 읽고, 미리 추출한 아이템 키워드 [{names_str}] 각각의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
summary에는 물건의 성능과 역할에 대해 간략하게 적어야 합니다.
character에는 해당 사건의 메인 인물을 적어야 합니다.

# 본문
{txt}
"""+"""   
# JSON_Output_Format
{
"아이템 키워드1": {"설명":"해당 물건 관한 짧은 설명 (없으면 Unknown)",
"관련인물" :["관련된인물 (없으면 Unknown)"] },
"아이템 키워드2": {"설명":"해당 물건 관한 짧은 설명 (없으면 Unknown)",
"관련인물" :["관련된인물 (없으면 Unknown)"] }
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 관련인물은 사건에 등장하는 모든 인물이 아닌 아이템과 관련된 주역만 적어야 합니다. 주역이 없을 경우 빈 배열 '[]'로 처리하세요.
- 만약 키워드가 하나만 있을 경우에는 한 물건만 출력하세요.
- 제공한 키워드를 제외한 나머지는 임의로 추가하여 출력하지 마세요.
"""
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            for k, v in result.items():
                item[k] = v
        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
    return {**state,
           '물건' :item
           }

### 6.설정추출 - 단체

In [140]:
def group(state: State) -> State:
    lis = state.get('check')
    lis = lis['집단']
    txt = state.get('txt')
    group = state.get('집단',{})
    
    for i in range(0, len(lis), 3):
        chunk = lis[i:i+3]
        names_str = ", ".join(chunk)
        system_prompt=f"""# Role
귀하는 소설의 원문을 분석하여 집단 설정(group Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.
본문에서 해당 정보가 있다면 "Unknown"을 삭제하고 정보를 str형식으로 적으세요

# Task
제공된 소설 본문을 읽고, 미리 추출한 집단 키워드 [{names_str}] 각각의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
summary에는 집단의 역사와 목적 대해 간략하게 적어야 합니다.
character에는 해당 집단에 속해있는 인물을 적은 후 집단에서 역할을 괄호'()'안에 적으세요.

# 본문
{txt}
"""+"""   
# JSON_Output_Format
{
"집단 키워드1": {"설명":"집단에 관한 짧은 설명(없으면 Unknown)",
"구성원" :["구성원이름(역할)(없으면 Unknown)"] },
"집단 키워드2": {"설명":"집단에 관한 짧은 설명(없으면 Unknown)",
"구성원" :["구성원이름(역할)(없으면 Unknown)"] },
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 본문에 정보가 전혀 없는 항목만 "Unknown"으로 남겨두세요.
- 구성원은 대상을 특정하기 위한 수식어(예: '데마시아의 정치체계','총의 악마','OO왕국 기사단')를 제외한 수식어는(예: '비시르 함장' X, '비시르' O) 모두 제거하여 넣으세요.
- 만약 키워드가 하나만 있을 경우에는 한 집단만 출력하세요.
- 제공한 키워드를 제외한 나머지는 임의로 추가하여 출력하지 마세요.
"""
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            for k, v in result.items():
                group[k] = v

        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
    return {**state,
           '집단' :group
           }

### 추출한 설정 검수 & 분류

In [39]:
def comparison(state: State):
    check = state.get('check')
    work_id = state.get('work_id')
    user_id = state.get('user_id')
    uplist ={'인물':[],'세계':[],'장소':[],'사건':[],'물건':[],'집단':[]}
    editlist = {'인물':[],'세계':[],'장소':[],'사건':[],'물건':[],'집단':[]}
    errorlist = {'인물':[],'세계':[],'장소':[],'사건':[],'물건':[],'집단':[]}
    oldlist = {'인물':[],'세계':[],'장소':[],'사건':[],'물건':[],'집단':[]}
    for tag,val in check.items():
        for keyword in val:
            emb = select(tag,keyword,user_id,work_id)
            if (len(emb)==0):
                uplist[tag].append({keyword:state.get(tag).get(keyword),"ep_num":[state.get("ep_num")]}) 
                continue
            oldlist[tag].append(emb)
            i = state.get(tag).get(keyword)
            system_prompt= f"""#### 역할: 소설 설정 검수 전문가
        
### 상황
하나의 대상에 대하여 서로 다른 에피소드에서 뽑아온 설정이 있습니다. 해당 두 설정을 확인하고 설정이 어긋나는 경우가 존재하는지 찾아보세요.

### 분석할 설정
기존설정 : {emb[0][1]}

신규설정 : {i}

### 요청 사항
2. **상세 분석:** - **인과관계/상세화:** 
   - **결합:** 한쪽이 원인이고 다른 쪽이 결과이거나, 한쪽이 다른 쪽에서 없는 설명인 경우 '결합'로 판단합니다.
   - **충돌:** 예외 없이 물리적으로 불가능한 경우(예: 죽었는데 살아있음, 파란 눈인데 빨간 눈임)에만 '충돌'로 판단하세요.
    -**일치:** 설정이 추가되거나 보강되는 부분 없이 동일한 경우에는 '일치'로 판단하세요.
    
### 출력 형식
[결과: 충돌/결합/일치]
[판단사유: 판단사유]
        """
            msg = f"이거 설정이 맞을까?"     
            result = genai(system_prompt,msg)
            if "[결과: 충돌]" in result:
                errorlist[tag].append({keyword:result,"신규설정":i})
            elif "[결과: 결합]" in result:
                editlist[tag].append(emb)
            print(result)
    editlist = edit_setting(editlist,state)
    return {'충돌': errorlist, '설정 결합' : editlist,
            "신규 업로드": uplist, "기존설정" : oldlist}

In [40]:
def edit_setting(editlist, state:State):
    lis = []
    for tag,value in editlist.items():
        for emb in value:            
            vctid,oldset,episode = emb[0]
            keyword = next(iter(oldset))
            system_prompt= f"""#### 역할: 소설 설정 작성 전문가
            
    ### 상황
    하나의 대상에 대하여 서로 다른 에피소드에서 뽑아온 설정이 있습니다. 해당 두 설정을 확인하고 두 설정을 결합하세요.
    
    ### 분석할 설정
    기존설정 : {oldset}
    
    신규설정 : {keyword} : {state.get(tag).get(keyword)}
    
    ### 출력 형식
    설정 형식과 동일
    """
            msg = f"두 설정을 결합해줄래"     
            result = genai(system_prompt,msg)
            try:
                result = result.replace("json", "").replace("```", "").strip()
                result = json.loads(result)
            except:
                print("딕셔너리 변환 에러 : edit_setting")
                print(result)
                #state = charter(state)
            print(result)
            episode.append(state.get('ep_num',0))
            lis.append([vctid,result,episode])
    return lis

## 원문&설정 읽기 쓰기

In [41]:
def save_text(user_id, work_id, ep_num, txt):
    try:
        save_dir = f"./save_original/{user_id}/{work_id}"
        os.makedirs(save_dir, exist_ok=True)
        file_path = os.path.join(save_dir, f"{ep_num}.txt")
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(txt)
            f.close()
        return f"./save_original/{user_id}/{work_id}/{ep_num}.txt"
    except Exception as e:
        print("error occured: ", e)
        return f"failed"

def open_text(user_id, work_id, ep_num) -> str:
    try:
        file_path = f"./save_original/{user_id}/{work_id}/{ep_num}.txt"
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e: 
        print("error occured: ", e)
        return "failed"
        
def save_json(user_id, work_id,lore_id, data):
    try:
        save_dir = f"./save_setting/{userid}/{work_id}"
        os.makedirs(save_dir, exist_ok=True)
        file_path = os.path.join(save_dir, f"{lore_id}.json")
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        return f"./save_setting/{userid}/{work_id}/{lore_id}.json"
    except Exception as e:
        print("error occured: ", e)
        return "failed"

def open_json(user_id, work_id,lore_id) :
    try:
        file_path = f"./save_original/{user_id}/{work_id}/{lore_id}.json"
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e: 
        print("error occured: ", e)
        return "failed"

## Rang Graph

### 카테고리

In [42]:
cat_builder = StateGraph(State)
cat_builder.add_node("Categories",Categories)

cat_builder.set_entry_point("Categories")
cat_builder.add_edge("Categories",END)

cat = cat_builder.compile()

### 상세설정

In [43]:
builder = StateGraph(State)
builder.add_node("charter",charter)
builder.add_node("rule",rule)
builder.add_node("place",place)
builder.add_node("event",event)
builder.add_node("item",item)
builder.add_node("group",group)

builder.set_entry_point("charter")
builder.add_edge("charter","rule")
builder.add_edge("rule","place")
builder.add_edge("place","event")
builder.add_edge("event","item")
builder.add_edge("item","group")
builder.add_edge("group",END)

detail = builder.compile()


# FastAPI

In [44]:
app = FastAPI()
users : Dict[str, State] = {}
origins = ["https://ai0917-kt-aivle-shool-8th-bigprojec-eta.vercel.app",
          "hppt://localhost:8080"]
app.add_middleware(
CORSMiddleware,
allow_origins=origins,
allow_credentials=True,
allow_methods=["GET", "POST", "PUT", "DELETE"],  
allow_headers=origins,
)
@app.get("/")
def connect_check():
    return "AI 서버와 연결되었습니다."

#원문 저장 
@app.post("/novel_save")
async def process_novel_save(data : dict = Body(...)):
    user_id = data.get("user_id")
    work_id = data.get("work_id")
    ep_num = data.get("ep_num")
    txt = data.get("txt")
    return save_text(user_id, work_id, ep_num, txt)

# 원문 읽기
@app.get("/novel_read")
async def process_novel_read(
    user_id: str = Query(...), 
    work_id: str = Query(...), 
    ep_num: int = Query(...)
):
    # 기존 코드의 txt 변수가 정의되어 있지 않아 에러가 날 수 있으니 확인이 필요해요!
    return open_text(user_id, work_id, ep_num)

#설정 저장
@app.post("/setting_save")
async def process_setting_save(data : dict = Body(...)):
    user_id = data.get("user_id")
    work_id = data.get("work_id")
    lore_id = data.get("lore_id")
    setting = data.get("data")
    return save_json(user_id,work_id,lore_id,setting)

# 설정 읽기
@app.get("/setting_read")
async def process_setting_read(
    user_id: str = Query(...), 
    work_id: str = Query(...), 
    lore_id: str = Query(...)
):
    return open_json(user_id, work_id, lore_id)

#카테고리 추출 
@app.post("/categories")
async def process_cat(data : dict = Body(...)):
    global current_state
    initial_state : State = {
        "txt": open_text(data.get("user_id"),data.get("work_id"),data.get("ep_num")),
        "ep_num" : data.get("ep_num"),
        "subtitle" : data.get("subtitle"),
        "check" : dict(),
        "work_id" : data.get("work_id"),
        "user_id" : data.get("user_id"),
        "인물" : dict(),
        "세계" : dict(),
        "장소" : dict(),
        "사건" : dict(),
        "물건" : dict(),
        "집단" : dict(),
    }
    if initial_state['txt'] == "failed":
        return "Error 원문이 저장되어 있지 않습니다."
    userid = initial_state.get('work_id')
    result = cat.invoke(initial_state)
    users[userid] = result
    return result.get("check",{})

@app.post("/setting") # 임베딩 벡터 반환하는 코드도 작성하기
async def process_get_setting(data : dict = Body(...)):
    userid = data.get("work_id")
    categories = data.get("check", [])
    if userid not in users:    # 존재하는 세션인지 확인
        raise HTTPException(status_code=404, detail="세션이 만료되었거나 존재하지 않습니다.")
    user_state = users[userid]  #기존의 데이/터 가져오기
    user_state["check"] = categories
    user_state = detail.invoke(user_state)
    result = comparison(user_state)
    return result 

@app.post("/comparison") # 임베딩 벡터 반환하는 코드도 작성하기
async def process_get_comparison(data : dict = Body(...)):
    result = comparison(data)
    return result 

@app.post("/get_vector") # 임베딩 벡터 반환하는 코드도 작성하기
async def process_get_vector(data : dict = Body(...)):
    original = data.get("original")
    txt_original = f"{original}"
    return get_vector(txt_original)

#유사도 기반 vectorDB 검색
@app.post("/userQ")
async def process_userQ(data : dict = Body(...)):
    category = data.get("category")
    user_query = data.get("user_query")
    user_id = data.get("user_id")
    work_id = data.get("work_id")
    sim = data.get("sim")
    limit = data.get("limit")
    return userQuestion(category,user_query,user_id,work_id,sim,limit)


@app.post("/dbupsert")
async def process_dbupsert(data : dict = Body(...)):
    work_id = data.get('work_id')
    user_id = data.get('user_id')
    universe_id = data.get('universe_id')
    setting = data.get('setting')
    try:
        insert_setting = setting.get('신규 업로드')
        lorebooks_insert(universe_id,work_id,user_id,insert_setting)
        edit_settings = setting.get('설정 결합')
        lorebooks_update(edit_settings)
    except Exception as e:
        return e
    return "성공"



# test

In [141]:
# state = group(state)

{
"데막시아 제국": {
"설명": "개척된 우주에서 최고의 권위를 가진 세력으로, 막강한 위세와 기술력으로 절대적인 지배 체제를 유지하고 있다. 황제의 이름 아래 오디널과 장군들이 이끄는 군사력으로 평화를 유지하려 노력하지만, 다양한 종족과 통제에 저항하는 불온 세력으로 인해 체제 전복 행위가 끊임없이 일어나고 있다. 제국은 오라를 통제하고 기사단의 불법적인 기술을 저지하며 질서를 유지하기 위해 노력한다.",
"구성원": ["케인(오디널)", "황제 자르반 4세(황제)", "나쿠리(사령관)", "바서르(함장)"]
},
"기사단": {
"설명": "데막시아 제국에 저항하는 반역 집단으로, 오라를 이용해 생체를 해킹하는 불법적인 기술을 가지고 있다. 제국의 체제에 위협적인 존재로 여겨지며, 변두리 행성에서 활동이 저지되고 있다.",
"구성원": ["Unknown"]
},
"신디케이트": {
"설명": "악명 높은 범죄 집단으로, 데막시아 제국에 고압적이고 거만하며 저항해야 할 대상으로 인식된다.",
"구성원": ["Unknown"]
}
}
변환 성공!
{
"오디널": {
"설명": "황제 직속의 가장 뛰어난 이들로, 황제를 보좌하며 제국의 군사력과 지배 체제를 유지하는 역할을 한다. 황제의 명령에 따라 강경책을 시행하거나, 제국의 질서를 유지하기 위한 임무를 수행한다.",
"구성원": [
"케인(황제 직속 장교)",
"나쿠리(사령관)",
"키일로(전투병기)"
]
}
}
변환 성공!


In [142]:
# print(state['check']['집단'])
# print("-"*8)
# for k,v in state['집단'].items():
#     print(k,":",v)
#     print("-"*8)

['데막시아 제국', '기사단', '신디케이트', '오디널']
--------
데막시아 제국 : {'설명': '개척된 우주에서 최고의 권위를 가진 세력으로, 막강한 위세와 기술력으로 절대적인 지배 체제를 유지하고 있다. 황제의 이름 아래 오디널과 장군들이 이끄는 군사력으로 평화를 유지하려 노력하지만, 다양한 종족과 통제에 저항하는 불온 세력으로 인해 체제 전복 행위가 끊임없이 일어나고 있다. 제국은 오라를 통제하고 기사단의 불법적인 기술을 저지하며 질서를 유지하기 위해 노력한다.', '구성원': ['케인(오디널)', '황제 자르반 4세(황제)', '나쿠리(사령관)', '바서르(함장)']}
--------
기사단 : {'설명': '데막시아 제국에 저항하는 반역 집단으로, 오라를 이용해 생체를 해킹하는 불법적인 기술을 가지고 있다. 제국의 체제에 위협적인 존재로 여겨지며, 변두리 행성에서 활동이 저지되고 있다.', '구성원': ['Unknown']}
--------
신디케이트 : {'설명': '악명 높은 범죄 집단으로, 데막시아 제국에 고압적이고 거만하며 저항해야 할 대상으로 인식된다.', '구성원': ['Unknown']}
--------
오디널 : {'설명': '황제 직속의 가장 뛰어난 이들로, 황제를 보좌하며 제국의 군사력과 지배 체제를 유지하는 역할을 한다. 황제의 명령에 따라 강경책을 시행하거나, 제국의 질서를 유지하기 위한 임무를 수행한다.', '구성원': ['케인(황제 직속 장교)', '나쿠리(사령관)', '키일로(전투병기)']}
--------


In [146]:
# len(state['txt'])

5477